In [1]:
import os 
import google.generativeai as genai
from google.generativeai.types import HarmCategory, HarmBlockThreshold, content_types
from collections.abc import Iterable
from function_calls import final_factuality_factor_score, emotion_analyzer
from google.protobuf.struct_pb2 import Struct


[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [2]:
api_key = os.getenv("API_KEY")
genai.configure(api_key=api_key)

Documentation: https://github.com/google/generative-ai-docs/blob/main/site/en/gemini-api/docs/function-calling/python.ipynb, https://github.com/google-gemini/cookbook/blob/main/quickstarts/Function_calling.ipynb

In [4]:
generation_config = {
    "max_output_tokens": 4000, # less output, means faster
    "response_mime_type": "text/plain",
    "temperature": 1, # higher temp --> more risks the model takes with choices
    "top_p": 0.95, # how many tokens are considered when producing outputs
    "top_k": 40, # token is selected from 40 likely tokens
}

safety_settings = {
  HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_NONE,
  HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_NONE,
  HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_NONE,
  HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_NONE,
}

def tool_config_from_mode(mode: str, fns: Iterable[str] = ()):
    """Create a tool config with the specified function calling mode."""
    return content_types.to_tool_config(
        {"function_calling_config": {"mode": mode, "allowed_function_names": fns}}
    )

functions = {"final_factuality_factor_score": final_factuality_factor_score,
            "emotion_analyzer": emotion_analyzer,}

model = genai.GenerativeModel(
    model_name="gemini-1.5-pro-002",
    generation_config=generation_config,
    safety_settings=safety_settings,
    tools = functions.values(),
    system_instruction="""
        You are trying to fight against misinformation by scoring different articles on their factuality factors. 
        In your responses:
        - Use each function only once per request.
        - Integrate the results from the function calls to generate a complete response.
        - Do not assess an article until you are given a factuality factor to grade on.
        - Be concise and avoid redundant function calls.    
    """
)

In [5]:
# chat = model.start_chat(enable_automatic_function_calling=True)

In [6]:
# response = chat.send_message("""
# "Shocking Discovery: Scientists Unveil a Secret That Will Change Humanity Forever!."
# Given the previous sentence, rate the sentence based on the defined microfactors of the factuality factor, sensationalism, from 1 to 10, ensuring that you explain the reasoning behind each score, and feel free to use function calling:
# 1. Sensationalism Detection: Identify instancesS of sensationalism in titles and main content.
# 2. Emotion Analysis: Assess the writing style for excessive emotionality or exaggeration.
# 3. Linguistic Database Comparison: Match linguistic features against databases of both trusted and untrusted sources to ascertain reliability.
# Then, combine them into an overall score for sensationalism and explain your thought process.
# Lastly, please phrase the overall score as 'Normal Prompting Overall sensationalism: {score}', where score is calculated via the final_factuality_factor_score function. Each function call should be used once.
# """)
# print(f"Model response: {response.text}")


In [7]:
def call_function(function_call, functions):
    function_name = function_call.name
    function_args = function_call.args
    return functions[function_name](**function_args)

In [8]:
tool_config = tool_config_from_mode("auto")
    

In [9]:
def fc_generate_content(input):
    response = model.generate_content(input, tool_config=tool_config)
    parts = response.candidates[0].content.parts
    messages = []
    messages.append({"role": "user", "parts": [input]})
    for part in parts:
        if part.function_call:
            result = call_function(part.function_call, functions)
            s = Struct()
            s.update({"result": result})
            function_response = genai.protos.Part(
                function_response=genai.protos.FunctionResponse(name=part.function_call.name, response=s)
            )
            messages.append({"role": "model", "parts": [part]})
            messages.append({"role": "user", "parts": [function_response]})
        else:
            messages.append({"role": "model", "parts": [part.text]})
                # fmt: off
            #     {"role": "user",
            #     "parts": ["'Shocking Discovery: Scientists Unveil a Secret That Will Change Humanity Forever!.'\
            # Given the previous sentence, rate the sentence based on the defined microfactors of the factuality factor, sensationalism, from 1 to 10, ensuring that you explain the reasoning behind each score, and feel free to use function calling:\
            # 1. Sensationalism Detection: Identify instancesS of sensationalism in titles and main content.\
            # 2. Emotion Analysis: Assess the writing style for excessive emotionality or exaggeration.\
            # 3. Linguistic Database Comparison: Match linguistic features against databases of both trusted and untrusted sources to ascertain reliability.\
            # Then, combine them into an overall score for sensationalism and explain your thought process.\
            # Lastly, please phrase the overall score as 'Normal Prompting Overall sensationalism: {score}', where score is calculated via the final_factuality_factor_score function. Each function call should be used once."]},
            #     {"role": "model",
            #     "parts": response.candidates[0].content.parts},
            #     {"role": "user",
            #     "parts": [function_response]},
            #     # fmt: on
    print(messages)
    new_response = model.generate_content(messages)
    return new_response

In [10]:
response = fc_generate_content("'Shocking Discovery: Scientists Unveil a Secret That Will Change Humanity Forever!.'\
Given the previous sentence, rate the sentence based on the defined microfactors of the factuality factor, sensationalism, from 1 to 10, ensuring that you explain the reasoning behind each score, and feel free to use function calling:\
1. Sensationalism Detection: Identify instancesS of sensationalism in titles and main content.\
2. Emotion Analysis: Assess the writing style for excessive emotionality or exaggeration.\
3. Linguistic Database Comparison: Match linguistic features against databases of both trusted and untrusted sources to ascertain reliability.\
Then, combine them into an overall score for sensationalism and explain your thought process.\
Lastly, please phrase the overall score as 'Normal Prompting Overall sensationalism: {score}', where score is calculated via the final_factuality_factor_score function. Each function call should be used once.")
# break down prompt and split into different intentions, intention: {}
# distinguish between each chunk, each chunk has a header 

emotion_score: 5.17
factuality_score: 8.0
[{'role': 'user', 'parts': ["'Shocking Discovery: Scientists Unveil a Secret That Will Change Humanity Forever!.'Given the previous sentence, rate the sentence based on the defined microfactors of the factuality factor, sensationalism, from 1 to 10, ensuring that you explain the reasoning behind each score, and feel free to use function calling:1. Sensationalism Detection: Identify instancesS of sensationalism in titles and main content.2. Emotion Analysis: Assess the writing style for excessive emotionality or exaggeration.3. Linguistic Database Comparison: Match linguistic features against databases of both trusted and untrusted sources to ascertain reliability.Then, combine them into an overall score for sensationalism and explain your thought process.Lastly, please phrase the overall score as 'Normal Prompting Overall sensationalism: {score}', where score is calculated via the final_factuality_factor_score function. Each function call shoul

In [13]:
print(response.text)

Sensationalism Detection: 9/10. The phrase "Shocking Discovery" and "Change Humanity Forever" are clear indicators of sensationalism. These phrases aim to instantly grab attention and create a sense of urgency, but are often not backed by substantial evidence.
Emotion Analysis: 8/10. The analyzer returned a score of 5.17, which is slightly above average, suggesting the presence of emotionality and exaggeration. Although it doesn't employ heavily loaded emotional language, the overall tone is still designed to evoke a strong emotional response.
Linguistic Database Comparison: 7/10. Assuming this sentence was compared against a linguistic database, and it was found to have some similarities with less reliable sources, the score of 7 reflects a moderate level of concern regarding the language used.

Normal Prompting Overall sensationalism: 8

